# Regression Decision Tree

For implementing Decision Tree with continuous variables, you will be using the LibSVM data. Such a dataset contains many missing values but Decision Trees can perform will with missing data as well. Hence, you will be implementing Decision Tree on this dataset.

In [1]:
#Load the libraries required for loading libSVM files
from pyspark.mllib.util import MLUtils

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1592028465843_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
#Load the data
data = MLUtils.loadLibSVMFile(sc, 's3a://decisiontreespark/libsvm_data.txt')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#View the count of data
data.take(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[LabeledPoint(0.0, (692,[127,128,129,130,131,154,155,156,157,158,159,181,182,183,184,185,186,187,188,189,207,208,209,210,211,212,213,214,215,216,217,235,236,237,238,239,240,241,242,243,244,245,262,263,264,265,266,267,268,269,270,271,272,273,289,290,291,292,293,294,295,296,297,300,301,302,316,317,318,319,320,321,328,329,330,343,344,345,346,347,348,349,356,357,358,371,372,373,374,384,385,386,399,400,401,412,413,414,426,427,428,429,440,441,442,454,455,456,457,466,467,468,469,470,482,483,484,493,494,495,496,497,510,511,512,520,521,522,523,538,539,540,547,548,549,550,566,567,568,569,570,571,572,573,574,575,576,577,578,594,595,596,597,598,599,600,601,602,603,604,622,623,624,625,626,627,628,629,630,651,652,653,654,655,656,657],[51.0,159.0,253.0,159.0,50.0,48.0,238.0,252.0,252.0,252.0,237.0,54.0,227.0,253.0,252.0,239.0,233.0,252.0,57.0,6.0,10.0,60.0,224.0,252.0,253.0,252.0,202.0,84.0,252.0,253.0,122.0,163.0,252.0,252.0,252.0,253.0,252.0,252.0,96.0,189.0,253.0,167.0,51.0,238.0,253.0,253.0,190.0

In [4]:
#View the count of data
data.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

100

## Split data into train, test 

In [5]:
#Use randomSplit to split data into 70-30%
(trainingData, testData) = data.randomSplit([0.7,0.3])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
#View the count of training data
trainingData.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

71

In [7]:
#View the count of test data
testData.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

29

## Train Decision Tree Regression Model

In [8]:
#import the libraries required for Decision Tree
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
#Create a Decision Tree model
model = DecisionTree.trainRegressor(trainingData, categoricalFeaturesInfo={}, 
                           impurity= "variance", maxDepth=2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
#View the model created
model

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DecisionTreeModel regressor of depth 2 with 5 nodes

## Prediction for Test data

In [11]:
#Predict result based on model created
predictions = model.predict(testData.map(lambda x: x.features))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
#View the predictions
predictions.take(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0]

## Label & Predict output together

In [13]:
#Place the actual and predicted labels together to get a better understanding
labelAndPreds = testData.map(lambda x: x.label).zip(predictions)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
#View the actual and predicted labels together to get a better understanding
labelAndPreds.take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(0.0, 0.0), (1.0, 1.0), (0.0, 0.0), (0.0, 0.0), (1.0, 1.0)]

## Calculate MSE

In [15]:
#Calculate and view MSE value
mse_value = labelAndPreds.map(lambda x: (x[0]-x[1])*(x[0]-x[1])).sum()/float(testData.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
mse_value

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.034482758620689655

## Visualize Model in Debug Mode

In [17]:
#print tree using model.toDebugString()
print(model.toDebugString())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DecisionTreeModel regressor of depth 2 with 5 nodes
  If (feature 434 <= 79.5)
   If (feature 99 <= 35.0)
    Predict: 0.0
   Else (feature 99 > 35.0)
    Predict: 1.0
  Else (feature 434 > 79.5)
   Predict: 1.0